In [1]:
from turbStats import tilt_mat, corr_mat, get_r0
from simulator import Simulator
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import torch
import os

In [2]:
# Select device and set patameters
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('CPU')

D = 0.1    # aperture diameter 
r0 = 0.1  # fried parameter set to [0.1, 0.05, 0.1/3, 0.025, 0.02] for turbulence of [1,2,3,4,5]
turbulence_strength = D/r0
print(turbulence_strength)

# Generate correlation matrix for tilt. Do this once for each different turbulence parameter. 
L = 3000
im_size = 112 # image size in DB
tilt_mat(im_size, D, r0, L)

# Load Simulator
simulator = Simulator(turbulence_strength, im_size).to(device, dtype=torch.float32)

1.0


In [ ]:
# Apply turbulence to the entire dataset
database = 'agedb'
data_path = './images/{}'.format(database)
output_path = './images/{}_turbulence_{}'.format(database, turbulence_strength)
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
image_list = os.listdir(data_path)
for image_name in tqdm(image_list):
    image = plt.imread(os.path.join(data_path, image_name))

    image = image / 255.0
    if len(image.shape) == 3: 
        image = image.transpose((2,0,1))
    image = torch.tensor(image, device = device, dtype=torch.float32)

    # Note that the current version does not support batched images. Only one frame at a time. 
    out_image = simulator(image).detach().cpu().numpy()

    if len(out_image.shape) == 3: 
        out_image = out_image.transpose((1,2,0))

    # save image
    out_image_path = os.path.join(output_path, image_name)
    out_image = np.clip(out_image, 0, 1.0)
    plt.imsave(out_image_path, out_image)

 20%|██        | 2413/12000 [00:10<00:40, 238.11it/s]